<a href="https://colab.research.google.com/github/Augista/EAS_Classification-Data/blob/main/EASkomnumCLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import RandomOverSampler


In [181]:
data = 'EASKOMNUM.csv'

df = pd.read_csv(data)
print(df.head())
# Memeriksa kolom yang ada dalam DataFrame
print("Kolom dalam DataFrame:", df.columns)


   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   1       145   233    1        2      150      0      2.3      3   
1   67    1   4       160   286    0        2      108      1      1.5      2   
2   67    1   4       120   229    0        2      129      1      2.6      2   
3   37    1   3       130   250    0        0      187      0      3.5      3   
4   41    0   2       130   204    0        2      172      0      1.4      1   

  ca thal  num  
0  0    6    0  
1  3    3    2  
2  2    7    1  
3  0    3    0  
4  0    3    0  
Kolom dalam DataFrame: Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'],
      dtype='object')


In [182]:
# Mengonversi kolom yang memiliki nilai string menjadi numerik jika diperlukan
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("Data setelah konversi nilai string menjadi numerik:")
print(df.head())

Data setelah konversi nilai string menjadi numerik:
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   1       145   233    1        2      150      0      2.3      3   
1   67    1   4       160   286    0        2      108      1      1.5      2   
2   67    1   4       120   229    0        2      129      1      2.6      2   
3   37    1   3       130   250    0        0      187      0      3.5      3   
4   41    0   2       130   204    0        2      172      0      1.4      1   

    ca  thal  num  
0  0.0   6.0    0  
1  3.0   3.0    2  
2  2.0   7.0    1  
3  0.0   3.0    0  
4  0.0   3.0    0  


In [183]:
# Mengisi nilai yang hilang dengan KNN Imputer
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
print("Data setelah imputasi nilai yang hilang:")
print(df_imputed.head())

Data setelah imputasi nilai yang hilang:
    age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  \
0  63.0  1.0  1.0     145.0  233.0  1.0      2.0    150.0    0.0      2.3   
1  67.0  1.0  4.0     160.0  286.0  0.0      2.0    108.0    1.0      1.5   
2  67.0  1.0  4.0     120.0  229.0  0.0      2.0    129.0    1.0      2.6   
3  37.0  1.0  3.0     130.0  250.0  0.0      0.0    187.0    0.0      3.5   
4  41.0  0.0  2.0     130.0  204.0  0.0      2.0    172.0    0.0      1.4   

   slope   ca  thal  num  
0    3.0  0.0   6.0  0.0  
1    2.0  3.0   3.0  2.0  
2    2.0  2.0   7.0  1.0  
3    3.0  0.0   3.0  0.0  
4    1.0  0.0   3.0  0.0  


In [197]:
# Normalisasi data ke rentang 0-1
scaler = MinMaxScaler()
columns_to_normalize = df.columns.difference(['num'])
df_normalized = pd.DataFrame(scaler.fit_transform(df_imputed), columns=df.columns)
df_normalized['num'] = df['num']
print("Data setelah normalisasi:")
print(df_normalized.head())

Data setelah normalisasi:
        age  sex        cp  trestbps      chol  fbs  restecg   thalach  exang  \
0  0.708333  1.0  0.000000  0.481132  0.244292  1.0      1.0  0.603053    0.0   
1  0.791667  1.0  1.000000  0.622642  0.365297  0.0      1.0  0.282443    1.0   
2  0.791667  1.0  1.000000  0.245283  0.235160  0.0      1.0  0.442748    1.0   
3  0.166667  1.0  0.666667  0.339623  0.283105  0.0      0.0  0.885496    0.0   
4  0.250000  0.0  0.333333  0.339623  0.178082  0.0      1.0  0.770992    0.0   

    oldpeak  slope        ca  thal  num  
0  0.370968    1.0  0.000000  0.75    0  
1  0.241935    0.5  1.000000  0.00    2  
2  0.419355    0.5  0.666667  1.00    1  
3  0.564516    1.0  0.000000  0.00    0  
4  0.225806    0.0  0.000000  0.00    0  


In [198]:
# fitur dan label kita pisahkan dulu (asumsi kolom terakhir adalah label)
X = df_normalized.iloc[:, :-1]
print("Fitur (X) setelah dipisahkan:")
print(X.head())
y = df_normalized.iloc[:, -1]
print("Label (y) setelah dipisahkan:")
print(y.head())

Fitur (X) setelah dipisahkan:
        age  sex        cp  trestbps      chol  fbs  restecg   thalach  exang  \
0  0.708333  1.0  0.000000  0.481132  0.244292  1.0      1.0  0.603053    0.0   
1  0.791667  1.0  1.000000  0.622642  0.365297  0.0      1.0  0.282443    1.0   
2  0.791667  1.0  1.000000  0.245283  0.235160  0.0      1.0  0.442748    1.0   
3  0.166667  1.0  0.666667  0.339623  0.283105  0.0      0.0  0.885496    0.0   
4  0.250000  0.0  0.333333  0.339623  0.178082  0.0      1.0  0.770992    0.0   

    oldpeak  slope        ca  thal  
0  0.370968    1.0  0.000000  0.75  
1  0.241935    0.5  1.000000  0.00  
2  0.419355    0.5  0.666667  1.00  
3  0.564516    1.0  0.000000  0.00  
4  0.225806    0.0  0.000000  0.00  
Label (y) setelah dipisahkan:
0    0
1    2
2    1
3    0
4    0
Name: num, dtype: int64


In [199]:
# Konversi label menjadi tipe kategori
y = y.astype('int')
print("Label (y) setelah konversi ke tipe kategori:")
print(y.head())

Label (y) setelah konversi ke tipe kategori:
0    0
1    2
2    1
3    0
4    0
Name: num, dtype: int64


In [200]:
# Memisahkan data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)


Training set size: (242, 13)
Testing set size: (61, 13)


In [201]:
# Train the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)
y_true = y_test

In [202]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred) * 100
report = (classification_report(y_true,y_pred, zero_division=1))

print("Accuracy:", accuracy)
print("Classification Report:")
print(report)

print("Predicted values:")
print(y_pred)
print("True values:")
print(y_true)

Accuracy: 55.73770491803278
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.88      0.86        34
           1       0.18      0.25      0.21         8
           2       0.50      0.18      0.27        11
           3       0.00      0.00      0.00         8
           4       0.00      1.00      0.00         0

    accuracy                           0.56        61
   macro avg       0.30      0.46      0.27        61
weighted avg       0.58      0.56      0.55        61

Predicted values:
[0 1 0 0 0 0 4 0 0 0 0 2 3 0 4 0 3 0 0 3 0 0 0 1 0 0 0 1 0 2 1 0 1 0 1 1 4
 0 0 0 0 0 0 0 0 0 3 1 1 3 0 0 0 0 2 0 2 4 1 3 1]
True values:
69     1
300    3
220    0
134    0
7      0
      ..
1      2
23     3
56     1
75     0
252    0
Name: num, Length: 61, dtype: int64


In [151]:
print("Fitur pelatihan (X_train):")
print(X_train.head())
print("Label pelatihan (y_train):")
print(y_train.head())
print("Fitur pengujian (X_test):")
print(X_test.head())
print("Label pengujian (y_test):")
print(y_test.head())

Fitur pelatihan (X_train):
          age  sex        cp  trestbps      chol  fbs  restecg   thalach  \
184  0.645833  0.0  1.000000  0.603774  0.408676  0.0      1.0  0.687023   
19   0.416667  1.0  0.333333  0.339623  0.319635  0.0      0.0  0.763359   
118  0.708333  1.0  1.000000  0.339623  0.465753  1.0      1.0  0.465649   
41   0.229167  1.0  0.000000  0.433962  0.166667  0.0      0.0  0.816794   
59   0.458333  1.0  0.000000  0.292453  0.198630  0.0      1.0  0.412214   

     exang   oldpeak  slope        ca  thal  
184    0.0  0.000000    0.0  0.000000   0.0  
19     0.0  0.096774    0.0  0.000000   0.0  
118    1.0  0.290323    0.0  1.000000   1.0  
41     1.0  0.225806    0.0  0.000000   1.0  
59     1.0  0.225806    0.0  0.333333   0.0  
Label pelatihan (y_train):
184    0
19     0
118    0
41     0
59     0
Name: num, dtype: int64
Fitur pengujian (X_test):
          age  sex        cp  trestbps      chol  fbs  restecg   thalach  \
69   0.354167  1.0  0.666667  0.528302  0.